# Maping

In [2]:
import pandas as pd
import numpy as np

In [2]:
data1= pd.DataFrame({"food": ["bacon", "pulled pork", "bacon", "Pastrami", "corned beef",
                              "Bacon", "pastrami", "honey ham", "nova lox"],
                    "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

print(data1)

          food  ounces
0        bacon     4.0
1  pulled pork     3.0
2        bacon    12.0
3     Pastrami     6.0
4  corned beef     7.5
5        Bacon     8.0
6     pastrami     3.0
7    honey ham     5.0
8     nova lox     6.0


Supongamos que queremos agregar una columna indicando el animal del cual proviene el corte. Escribimos un mapping:

In [4]:
meat_to_animal= {"bacon": "pig", "pulled pork": "pig", "pastrami": "cow", 
                 "corned beef": "cow", "honey ham": "pig", "nova lox": "salmon"}

print(meat_to_animal) #Básicamente es un diccionario

{'bacon': 'pig', 'pulled pork': 'pig', 'pastrami': 'cow', 'corned beef': 'cow', 'honey ham': 'pig', 'nova lox': 'salmon'}


El primer problema que encontramos es que en los datos una parte está con mayúscula, en otra con minúscula. En consecuencia, lo primero que hacemos es transformar todo a minúscula:

In [8]:
data2= data1["food"].str.lower()

print(data2)

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object


Ahora procedemos con el join, agregamos una columna nueva indicando el animal:

In [13]:
data1["animal"]= data2.map(meat_to_animal) 

print(data1)

          food  ounces  animal
0        bacon     4.0     pig
1  pulled pork     3.0     pig
2        bacon    12.0     pig
3     Pastrami     6.0     cow
4  corned beef     7.5     cow
5        Bacon     8.0     pig
6     pastrami     3.0     cow
7    honey ham     5.0     pig
8     nova lox     6.0  salmon


# If - Then

Esto se hace con numpy, se puede hacer con técnica pandas, pero es mucho más rápido por numpy.

Importamos el archivo a usar:

In [3]:
xlsx= pd.ExcelFile('examples\Inventario_202008.xlsx')

File1= pd.read_excel(xlsx)

print(File1.head())

   SISTEMA  SUCURS  LINEA_TIPO  OPERCTA  CODGARORIG  CODGAR  PRODGARANT  \
0        4       1           0   332877           0       0           0   
1        4       1           0   336444           0       0           0   
2        4       1           0   340698           0       0           0   
3        4       1           0   342689           0       0           0   
4        4       1           0   344224           0       0           0   

   GTIAPREFER  CODIDENT     NROIDENT  ...  LIMITE FLAG_USAR_ RATIO_PREV  \
0         0.0        11  27278296038  ...       0          0          0   
1         0.0        11  27361059811  ...       0          0          0   
2         0.0        11  27112138469  ...       0          0          0   
3         0.0        11  27180636833  ...       0          0          0   
4         0.0        11  27282101926  ...       0          0          0   

  SALDO_EAD PREVISION_ CTA_ACTIVO  CTA_SDO_NO  PPOPEESN9  PVAINCON9  PVAPVCON9  
0         0      

Establecemos primero las condiciones a evaluar. Generara una serie con valor verdadero o falso dependiendo si se cumple o no la condición:

In [27]:
conditions= [((File1["PRODUC_SFB"] == 925) & (File1["CODSIT"] == "1"))]

print(conditions)

[0         False
1         False
2         False
3         False
4         False
          ...  
125816    False
125817    False
125818    False
125819    False
125820    False
Length: 125821, dtype: bool]


Luego establecemos con un tupple, el valor que adoptaran aquellas condiciones que sean Verdaderas

In [28]:
values= ["Gabriel"]

Finalmente, juntamos el valor con las condiciones True. Las que cumplan las condiciones, recibirán el valor buscado. Las que no, recibirán 0.

In [29]:
File1['Nueva_Columna'] = np.select(conditions, values)

print(File1)

        SISTEMA  SUCURS  LINEA_TIPO  OPERCTA  CODGARORIG  CODGAR  PRODGARANT  \
0             4       1           0   332877           0       0           0   
1             4       1           0   336444           0       0           0   
2             4       1           0   340698           0       0           0   
3             4       1           0   342689           0       0           0   
4             4       1           0   344224           0       0           0   
...         ...     ...         ...      ...         ...     ...         ...   
125816       99      85           0   434974           0       0           0   
125817       99      85           0   444797           0       0           0   
125818       99      85           0   452018           0       0           0   
125819       99      85           0   455124           0       0           0   
125820        4       1           0   457129           0       0           0   

        GTIAPREFER  CODIDENT     NROIDE

Verificamos:

In [42]:
File2= File1[File1['Nueva_Columna']== "Gabriel"]

print(File2)

       SISTEMA  SUCURS  LINEA_TIPO  OPERCTA  CODGARORIG  CODGAR  PRODGARANT  \
331          4       1           0   304806           0       0           0   
332          4       1           0   304811           0       0           0   
333          4       1           0   305001           0       0           0   
334          4       1           0   305139           0       0           0   
335          4       1           0   305140           0       0           0   
...        ...     ...         ...      ...         ...     ...         ...   
89297       18      85           0   455223           0       0           0   
89298       18      85           0   455272           0       0           0   
89299       18      85           0   455321           0       0           0   
89300       18      85           0   455350           0       0           0   
89301       18      85           0   455450           0       0           0   

       GTIAPREFER  CODIDENT     NROIDENT  ...  FLAG

Ahora vamos a ver cómo hacemos si tenemos varias condiciones:

In [47]:
conditions1= [((File1["PRODUC_SFB"] == 925) & (File1["CODSIT"] == "1")),
              ((File1["PRODUC_SFB"] == 925) & (File1["CODSIT"] == "2A")),
              ((File1["PRODUC_SFB"] == 925) & (File1["CODSIT"] == "3")),
              ((File1["PRODUC_SFB"] == 925) & (File1["CODSIT"] == "4")),
              ((File1["PRODUC_SFB"] == 925) & (File1["CODSIT"] == "5")),
              ((File1["PRODUC_SFB"] == 925) & (File1["CODSIT"] == "6"))]

values1= ["Gabriel_1","Gabriel_2","Gabriel_3","Gabriel_4","Gabriel_5","Gabriel_6"]

File1['Nueva_Columna'] = np.select(conditions1, values1)

#print(conditions1)

Lo que hace básicamente es generar cuantas tablas(series) como condiciones haya. Las condiciones se separan por coma. Entonces en la primera condición, se debe cumplir que sea producto 925 y además el codsit sea igual a 1. Genera una tabla por esto, y así sucesivamente por cada condición. Para ver esto, imprimir la tabla conditions1.

In [53]:
File2= File1[File1['Nueva_Columna'] == "Gabriel_1"]

print(File2)

       SISTEMA  SUCURS  LINEA_TIPO  OPERCTA  CODGARORIG  CODGAR  PRODGARANT  \
331          4       1           0   304806           0       0           0   
332          4       1           0   304811           0       0           0   
333          4       1           0   305001           0       0           0   
334          4       1           0   305139           0       0           0   
335          4       1           0   305140           0       0           0   
...        ...     ...         ...      ...         ...     ...         ...   
89297       18      85           0   455223           0       0           0   
89298       18      85           0   455272           0       0           0   
89299       18      85           0   455321           0       0           0   
89300       18      85           0   455350           0       0           0   
89301       18      85           0   455450           0       0           0   

       GTIAPREFER  CODIDENT     NROIDENT  ...  FLAG

Podemos también mezclar condiciones de "y" y "o":

In [54]:
conditions2= [((File1["PRODUC_SFB"] == 925) & (File1["CODSIT"] == "1")) |
              ((File1["PRODUC_SFB"] == 926) & (File1["CODSIT"] == "1")),
              ((File1["PRODUC_SFB"] == 925) & (File1["CODSIT"] == "2A")) |
              ((File1["PRODUC_SFB"] == 926) & (File1["CODSIT"] == "2A")),
              ((File1["PRODUC_SFB"] == 925) & (File1["CODSIT"] == "3")) |
              ((File1["PRODUC_SFB"] == 926) & (File1["CODSIT"] == "3")),
              ((File1["PRODUC_SFB"] == 925) & (File1["CODSIT"] == "4")) |
              ((File1["PRODUC_SFB"] == 926) & (File1["CODSIT"] == "4")),
              ((File1["PRODUC_SFB"] == 925) & (File1["CODSIT"] == "5")) |
              ((File1["PRODUC_SFB"] == 926) & (File1["CODSIT"] == "5")),
              ((File1["PRODUC_SFB"] == 925) & (File1["CODSIT"] == "6")) |
              ((File1["PRODUC_SFB"] == 926) & (File1["CODSIT"] == "6"))]

values2= ["Gabriel_1","Gabriel_2","Gabriel_3","Gabriel_4","Gabriel_5","Gabriel_6"]

File1['Nueva_Columna'] = np.select(conditions2, values2)

#print(conditions1)

Controlamos:

In [56]:
File3= File1[File1['PRODUC_SFB'] == 925]

print(File3)

       SISTEMA  SUCURS  LINEA_TIPO  OPERCTA  CODGARORIG  CODGAR  PRODGARANT  \
331          4       1           0   304806           0       0           0   
332          4       1           0   304811           0       0           0   
333          4       1           0   305001           0       0           0   
334          4       1           0   305139           0       0           0   
335          4       1           0   305140           0       0           0   
...        ...     ...         ...      ...         ...     ...         ...   
89297       18      85           0   455223           0       0           0   
89298       18      85           0   455272           0       0           0   
89299       18      85           0   455321           0       0           0   
89300       18      85           0   455350           0       0           0   
89301       18      85           0   455450           0       0           0   

       GTIAPREFER  CODIDENT     NROIDENT  ...  FLAG